In [1]:
import scanpy as sc

from scprint import scPrint
from scprint.tasks import Embedder, Denoiser, GNInfer

from scdataloader import Preprocessor
from scdataloader.utils import load_genes, translate

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier

from scdataloader.utils import get_descendants
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection

from collections import Counter
from scdataloader.utils import downsample_profile
from scipy.stats import spearmanr

from scprint.tasks import GNInfer

from bengrn import BenGRN

%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')

→ connected lamindb: jkobject/scprint2


In [2]:
m = torch.load('../../xzp23r4p.ckpt', map_location=torch.device('cpu'))

In [3]:
model = scPrint.load_from_checkpoint(
    '../../xzp23r4p.ckpt', precpt_gene_emb=None, classes=m['hyper_parameters']['label_counts'], 
)#strict=False)
    #transformer="normal")
del m

FYI: scPrint is not attached to a `Trainer`.


## Loading the data

In [ ]:
adata = sc.read(
    "data/pancreas_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539828",
)


In [5]:
adata = sc.read(
    "data/human_brain_smartv4.h5ad",
    backup_url="https://datasets.cellxgene.cziscience.com/f156b755-78c7-4928-917a-be138c153b85.h5ad",
)


  0%|          | 0.00/2.47G [00:00<?, ?B/s]

In [8]:
adata.X.sum()

66897050000.0

In [ ]:
preprocessor = Preprocessor(force_preprocess=True, skip_validate=True,
                            do_postp=False, min_valid_genes_id=5000, min_dataset_size=64)
nadata = preprocessor(adata.copy())

Dropping layers:  KeysView(Layers with keys: exon, intron)
checking raw counts
removed 0 non primary cells, 49417 renamining
filtered out 0 cells, 49417 renamining
Removed 0 genes.
startin QC
Seeing 10181 outliers (20.60% of total dataset):
done
AnnData object with n_obs × n_vars = 49417 × 70704
    obs: 'suspension_type', 'cluster', 'class', 'subclass', 'sex_ontology_term_id', 'region', 'cortical_layer', 'cell_type_accession', 'cell_type_alias', 'cell_type_alt_alias', 'cell_type_designation', 'donor_id', 'outlier_call', 'outlier_type', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'assay_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'cell_type_ontology_term_id', 'Specimen ID', 'sample_tissue_type', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'cause_of_death', 'PMI (hr)', 'Tissue (RIN)', 'Hemisphere Sampled', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'develo

AnnData object with n_obs × n_vars = 49417 × 35793
    obs: 'suspension_type', 'cluster', 'class', 'subclass', 'sex_ontology_term_id', 'region', 'cortical_layer', 'cell_type_accession', 'cell_type_alias', 'cell_type_alt_alias', 'cell_type_designation', 'donor_id', 'outlier_call', 'outlier_type', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'assay_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'cell_type_ontology_term_id', 'Specimen ID', 'sample_tissue_type', 'development_stage_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'cause_of_death', 'PMI (hr)', 'Tissue (RIN)', 'Hemisphere Sampled', 'tissue_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'citation', 'cortical_layer_colors', 'region_colors', 'schema_reference', 'schema

# training

In [17]:
from scdataloader import DataModule

In [18]:
gene_emb = '../data/main/gene_embeddings.parquet'
datamodule = DataModule(
    collection_name="scPRINT-V2 test", #some, all, preprocessed dataset, all no zhang, 
    gene_embeddings=gene_emb,
    clss_to_weight=[],
    metacell_mode=0,
    clss_to_predict=[],
    hierarchical_clss=[],
    organisms=["NCBITaxon:9606", "NCBITaxon:10090"],
    how="random expr",
    max_len=10_000,
    add_zero_genes=0,
    # how much more you will see the most present vs less present category
    weight_scaler=0,
    batch_size=1,
    num_workers=3,
    prefetch_factor=3,
    n_samples_per_epoch=2000,
    validation_split=0.05,
    do_gene_pos='../data/main/biomart_pos.parquet',
    pin_memory=True,
    test_split=0.05)
testfiles = datamodule.setup()

! no run & transform got linked, call `ln.track()` & re-run
! run input wasn't tracked, call `ln.track()` and re-run
won't do any check but we recommend to have your dataset coming from local storage


/home/ml4ig1/Documents code/scDataLoader/scdataloader/utils.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)


100.0% are aligned
seeing a string: loading gene positions as biomart parquet file


KeyError: 'organism_ontology_term_id'

# testing

In [4]:
adata = sc.read(
    "data/lung_smart2.h5ad",
    backup_url="https://datasets.cellxgene.cziscience.com/6ebba0e0-a159-406f-8095-451115673a2c.h5ad",
)

In [33]:
preprocessor = Preprocessor(force_preprocess=True, skip_validate=True,
                            do_postp=False, min_valid_genes_id=5000, min_dataset_size=64)
adata = preprocessor(adata.copy())

Dropping layers:  KeysView(Layers with keys: true)
checking raw counts
removed 0 non primary cells, 9409 renamining
filtered out 0 cells, 9409 renamining
Removed 0 genes.
startin QC
Seeing 3903 outliers (41.48% of total dataset):
done
AnnData object with n_obs × n_vars = 9409 × 70704
    obs: 'nGene', 'nReads', 'plate.barcode', 'cell.id', 'region', 'label', 'sorter', 'sort.location', 'sample', 'location', 'donor_id', 'percent.ercc', 'percent.ribo', 'gating', 'free_annotation', 'Number of splices: Total', 'Number of splices: Annotated (sjdb)', 'Number of splices: GT-AG', 'Number of splices: GC-AG', 'Number of splices: AT-AC', 'Number of splices: Non-canonical', 'Mapping speed, Million of reads per hour', 'Average input read length', 'compartment', 'tissue_ontology_term_id', 'disease_ontology_term_id', 'development_stage_ontology_term_id', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'is_primary_data', 'organi

In [20]:
adata.layers['true'] = adata.X.copy()

In [ ]:
# denoising without
denoise = Denoiser(
    batch_size=10,
    max_len=10_000,
    max_cells=10_000,
    doplot=False,
    num_workers=4,
    predict_depth_mult=5,
    how="most var",
    downsample_expr=0.7,
)
metrics, idx, nadata = denoise(model, adata)
metrics

/home/ml4ig1/Documents code/scDataLoader/scdataloader/utils.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)
100%|██████████| 100/100 [00:10<00:00,  9.20it/s]


AnnData object with n_obs × n_vars = 1000 × 44756
    obs: 'pred_cell_type_ontology_term_id', 'pred_tissue_ontology_term_id', 'pred_disease_ontology_term_id', 'pred_age_group', 'pred_assay_ontology_term_id', 'pred_self_reported_ethnicity_ontology_term_id', 'pred_sex_ontology_term_id', 'pred_organism_ontology_term_id', 'conv_pred_cell_type_ontology_term_id', 'conv_pred_tissue_ontology_term_id', 'conv_pred_disease_ontology_term_id', 'conv_pred_age_group', 'conv_pred_assay_ontology_term_id', 'conv_pred_self_reported_ethnicity_ontology_term_id'
    obsm: 'scprint_emb'
    layers: 'scprint_mu', 'scprint_theta', 'scprint_pi'


{'reco2noisy': 0.2372157155812925,
 'reco2full': 0.8119446951655541,
 'noisy2full': 0.37784544327120057}

In [22]:
nadata = adata[:, adata.var.index.isin(model.genes)]

In [23]:
sc.pp.highly_variable_genes(
    nadata, flavor="seurat_v3", n_top_genes=10_000, span=0.99)

In [29]:
subadata = nadata[:, nadata.var.highly_variable].copy()
subadata.X = downsample_profile(torch.Tensor(subadata.X.toarray()), 0.7)


In [30]:
denoised_adata = sc.external.pp.magic(subadata.copy(), copy=True, n_jobs=10, solver='approximate', verbose=True)

Calculating MAGIC...
  Running MAGIC on 9409 cells and 10000 genes.
  Running MAGIC on 9409 cells and 10000 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 4.13 seconds.
    Calculating KNN search...
    Calculated KNN search in 3.73 seconds.
    Calculating affinities...
    Calculated affinities in 3.93 seconds.
  Calculated graph and diffusion operator in 11.83 seconds.
  Calculating imputation...
  Calculated imputation in 0.10 seconds.
Calculated MAGIC in 14.15 seconds.


In [31]:
reco = denoised_adata.X
true = subadata.layers['true'].toarray()
noisy = subadata.X
corr_coef, p_value = spearmanr(
    np.vstack(
        [reco[true != 0], noisy[true != 0], true[true != 0]]).T
)
{
    "reco2noisy": corr_coef[0, 1],
    "reco2full": corr_coef[0, 2],
    "noisy2full": corr_coef[1, 2],
}

{'reco2noisy': 0.2212071641243409,
 'reco2full': 0.5186957772724994,
 'noisy2full': 0.41001806749787767}

In [ ]:
# denoising

In [36]:
adata.obs["cell_type"].value_counts()

cell_type
natural killer cell                                     1232
club cell                                                898
capillary endothelial cell                               872
endothelial cell                                         816
pulmonary alveolar type 2 cell                           760
lung ciliated cell                                       551
fibroblast                                               533
pericyte                                                 518
pulmonary alveolar type 1 cell                           421
naive thymus-derived CD8-positive, alpha-beta T cell     378
bronchial smooth muscle cell                             346
respiratory basal cell                                   240
lung goblet cell                                         233
epithelial cell                                          201
vascular associated smooth muscle cell                   176
naive thymus-derived CD4-positive, alpha-beta T cell     131
B cell        

In [ ]:
# gene network inference without
metrics = {}
clf_omni = None
shapes = []
for celltype in ['natural killer cell', 'fibroblast', 'B cell ']:
    grn_inferer = GNInfer(
                        how="most var",
                        preprocess="softmax",
                        head_agg='mean',
                        filtration="none",
                        forward_mode="none",
                        num_genes=10_000,
                        max_cells=1000,
                        doplot=False,
                        batch_size=10,
                        cell_type_col="cell_type",
                        layer=list(range(model.nlayers))[:]
                        )
    grn = grn_inferer(model, adata, cell_type=celltype)
    grn.var.index = make_index_unique(grn.var['symbol'].astype(str))
    metrics[celltype+'_scprint_full'] = BenGRN(grn).scprint_benchmark()
    grn.write_h5ad(f'../data/temp/grn_{celltype}_scprint_f.h5ad')

In [ ]:
# gene network inference

In [ ]:
# embedding and cell type prediction with

In [ ]:
# embedding and cell type prediction without